 # Google Earth Engine Exploration

In [5]:
import pandas as pd
import ee
import geopandas as gpd
from IPython.display import Image
import folium

In [10]:
# Autenticate (Needs to be done once a week)
ee.Authenticate()



Successfully saved authorization token.


In [6]:
# Initialise to make sure credentials are correct
ee.Initialize()

In [7]:
# Print name of collection to make sure can access the data
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

NASADEM: NASA NASADEM Digital Elevation 30m


# Testing Earth Engine is Working Using Demo

In [8]:
# Get the elevation at mt everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()

print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [9]:
# Create  A mask for all points where it is not greater than 0
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [10]:
# Functiono to add Earth Engine Layer to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)


# Combining Earth Engine with RHoMIS and IPUMS

In [11]:
# Read in RHoMIS and convert to GEO Data Frame
rhomis_data = pd.read_csv("./data/prepared-data/rhomis-geo-data.csv", encoding="latin", low_memory=False)
rhomis_data = gpd.GeoDataFrame(rhomis_data) 
rhomis_data['geometry'] = gpd.GeoSeries.from_wkt(rhomis_data['geometry'])

# Read in IPUMS and convert to GEO Data Frame
ipums_data = pd.read_csv("./data/prepared-data/ipums-all.csv", encoding="latin", low_memory=False) 
ipums_data = gpd.GeoDataFrame(ipums_data)
ipums_data['geometry'] = gpd.GeoSeries.from_wkt(ipums_data['geometry'])


In [17]:
# Plot RHoMIS Points on FOLIUM Map

map = folium.Map(location = [0.498514,25.659943], tiles='OpenStreetMap' , zoom_start = 4)

rhomis_geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in rhomis_data.head(1000).geometry ]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in rhomis_geo_df_list:
    #assign a color marker for the type of volcano, Strato being the most common
    # if rhomis_data.Type[i] == "Stratovolcano":
    #     type_color = "green"
    # elif rhomis_data.Type[i] == "Complex volcano":
    #     type_color = "blue"
    # elif rhomis_data.Type[i] == "Shield volcano":
    #     type_color = "orange"
    # elif rhomis_data.Type[i] == "Lava dome":
    #     type_color = "pink"
    # else:
    #     type_color = "purple"


    # Place the markers with the popup labels and data
    map.add_child(folium.Marker(location = coordinates,
                            popup =
                            "Country: " + str(rhomis_data.iso_2[i]) + '<br>' +
                            "HFIAS: " + str(rhomis_data.hfias_status[i]),
                            icon = folium.Icon(color = "blue")))
    i = i + 1

display(map)


In [2]:
# Plot Polygons

for _, r in ipums_data.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)
m

range(0, 10)

In [125]:
# Sample point from RHoMIS
scale = 1000
[lat, lon] = rhomis_data.loc[1,["gps_lat", "gps_lon"]]
point = ee.Geometry.Point([lat, lon])


In [163]:
# Sample polygon from IPUMS
polygon = ipums_data.loc[1,"geometry"]
polygon = [point for point in polygon.boundary.coords]
polygon = ee.Geometry.Polygon(polygon)

In [215]:
# Import the MODIS land cover collection.
# And Extract the appropriate band ()
lc_image_collection = ee.ImageCollection('MODIS/006/MCD12Q1')
lc_image_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'MODIS/006/MCD12Q1',
 'version': 1641990386671551,
 'properties': {'system:visualization_0_min': '1.0',
  'type_name': 'ImageCollection',
  'keywords': ['landcover', 'mcd12q1', 'modis', 'nasa', 'usgs', 'yearly'],
  'thumb': 'https://mw1.google.com/ges/dd/images/MODIS_006_MCD12Q1_thumb.png',
  'description': '<p>The MCD12Q1 V6 product provides global land cover types at yearly intervals\n(2001-2016) derived from six different classification schemes. It is derived\nusing supervised classifications of MODIS Terra and Aqua reflectance data.\nThe supervised classifications then undergo additional post-processing that\nincorporate prior knowledge and ancillary information to further refine\nspecific classes.</p><p>Documentation:</p><ul><li><p><a href="https://lpdaac.usgs.gov/documents/101/MCD12_User_Guide_V6.pdf">User&#39;s Guide</a></p></li><li><p><a href="https://lpdaac.usgs.gov/documents/86/MCD12_ATBD.pdf">Algorithm Theoretical Basis Docume

In [219]:
# Get an image of the particular Date
lc_image_collection = (lc_image_collection.
# Select the band
select("LC_Type1").
# Filter Date
filterDate('2018-12-31', '2019-01-01'))

# Print Metadata for image
lc_image_collection.getInfo()
# Getting a stack of images for that year
#lc_image_mode = lc_image.mode()


#print(image)

{'type': 'ImageCollection',
 'bands': [],
 'id': 'MODIS/006/MCD12Q1',
 'version': 1641990386671551,
 'properties': {'system:visualization_0_min': '1.0',
  'type_name': 'ImageCollection',
  'keywords': ['landcover', 'mcd12q1', 'modis', 'nasa', 'usgs', 'yearly'],
  'thumb': 'https://mw1.google.com/ges/dd/images/MODIS_006_MCD12Q1_thumb.png',
  'description': '<p>The MCD12Q1 V6 product provides global land cover types at yearly intervals\n(2001-2016) derived from six different classification schemes. It is derived\nusing supervised classifications of MODIS Terra and Aqua reflectance data.\nThe supervised classifications then undergo additional post-processing that\nincorporate prior knowledge and ancillary information to further refine\nspecific classes.</p><p>Documentation:</p><ul><li><p><a href="https://lpdaac.usgs.gov/documents/101/MCD12_User_Guide_V6.pdf">User&#39;s Guide</a></p></li><li><p><a href="https://lpdaac.usgs.gov/documents/86/MCD12_ATBD.pdf">Algorithm Theoretical Basis Docume

In [213]:
# Get a point reading


# Print the elevation of the sample point
lc_point = lc_image.mode().sample(point, scale)#.first().get("LC_Type1").getInfo()
print('Land Cover Class at point is:', lc_point)

Land Cover Class at point is: ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Image.sample",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "reduce.mode",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlgorithm": {
                          "functionDefinitionValue": {
                            "argumentNames": [
                              "_MAPPING_VAR_0_0"
                            ],
                            "body": {
                              "functionInvocationValue": {
                                "functionName": "Image.select",
                        

In [167]:

# Import the Landsat 8 TOA image collection.

# Get the least cloudy image in 2015.
image = ee.Image(
  lc.filterBounds(point)
    .filterDate('2015-01-01', '2015-01-01')
    .first()
)

median = image.reduce(ee.Reducer.count())

print(median)


ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.reduce",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.load",
                            "arguments": {
                              "id": {
                                "constantValue": "MODIS/006/MCD12Q1"
                              }
                            }
                          }
                        },
              

In [39]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
land_cover_point = lc.sample(point, scale)#.first().get('elevation').getInfo()
print('Ground elevation at urban point:', land_cover_point, 'm')


AttributeError: 'ImageCollection' object has no attribute 'sample'

In [40]:
lc

In [44]:
dir(lc)

['And',
 'Or',
 '_HAS_DYNAMIC_ATTRIBUTES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_preparation_function',
 '_cast',
 '_getThumbURL',
 '_initialized',
 'aggregate_array',
 'aggregate_count',
 'aggregate_count_distinct',
 'aggregate_first',
 'aggregate_histogram',
 'aggregate_max',
 'aggregate_mean',
 'aggregate_min',
 'aggregate_product',
 'aggregate_sample_sd',
 'aggregate_sample_var',
 'aggregate_stats',
 'aggregate_sum',
 'aggregate_total_sd',
 'aggregate_total_var',
 'args',
 'aside',
 'cache',
 'cast',
 'combine',
 'copyProperties',
 'count',
 'distance',
 'distinct',
 'draw',
 'elementType',
 'encode',
 'encode_cloud_value',
 'errorMatrix',
 'filte